In [5]:
import cv2
import os
import numpy as np
import dlib
from mtcnn.mtcnn import MTCNN

# Load the MTCNN model
detector = MTCNN()

# Load the landmark predictor
predictor = dlib.shape_predictor("../HAAR_Models/Shape Predictor/shape_predictor_68_face_landmarks.dat")

# Create a directory to save the cropped and aligned faces
os.makedirs("aligned_faces4", exist_ok=True)

# Define the standard name format for the cropped faces
face_name_format = "face_{}_{}.jpg"

# Counter to keep track of the face number
face_counter = 1

# Margin to add around the face
margin = 30

# Function to align the face
def align_face(pixels, landmarks):
    # Get the center point between the two eyes
    eye_center = ((landmarks[0, 0] + landmarks[1, 0]) // 2,
                  (landmarks[0, 1] + landmarks[1, 1]) // 2)

    # Get the angle between the eye center and the nose
    angle = np.degrees(np.arctan2(landmarks[1, 1] - landmarks[0, 1],
                                  landmarks[1, 0] - landmarks[0, 0]))

    # Rotate the image by the angle
    rot_mat = cv2.getRotationMatrix2D(eye_center, angle, scale=1.0)
    result = cv2.warpAffine(pixels, rot_mat, pixels.shape[1::-1], flags=cv2.INTER_CUBIC)

    return result

# Loop through all the images in the folder
for filename in os.listdir("../dataset/Face/Group 4"):
    # Load the input image
    try:
        pixels = cv2.imread(f"../dataset/Face/Group 4/{filename}")
    except:
        print(f"Skipping {filename} - not a valid image")
        continue

    # Detect faces in the image
    try:
        faces = detector.detect_faces(pixels)
    except Exception as e:
        print(f"An error occured while detecting faces in {filename}: {e}")
        continue

    # If no faces are found in the image, skip to the next image
    if len(faces) == 0:
        print(f"No faces found in {filename}, skipping to next image")
        continue

    # Crop and align the faces
    for i, face in enumerate(faces):
        x, y, w, h = face['box']
        x1 = max(0, x - margin)
        y1 = max(0, y - margin)
        x2 = x + w + margin
        y2 = y + h + margin
        face_pixels = pixels[y1:y2, x1:x2]

        # Get the landmarks for the face
        landmarks = face['keypoints']
        landmarks = np.array([(landmarks['left_eye'][0], landmarks['left_eye'][1]),
                             (landmarks['right_eye'][0], landmarks['right_eye'][1]),
                             (landmarks['nose'][0], landmarks['nose'][1]),
                             (landmarks['mouth_left'][0], landmarks['mouth_left'][1]),
                             (landmarks['mouth_right'][0], landmarks['mouth_right'][1])],
                            dtype=np.float32)

        # Align the face
        face_pixels = align_face(face_pixels, landmarks)

        # Save the face with a standardized new name
        face_pixels = cv2.resize(face_pixels, (224, 224))
        face_name = face_name_format.format(face_counter, i)
        cv2.imwrite(f"aligned_faces4/{face_name}", face_pixels)

    # Increment the face counter for the next image
    face_counter += 1

1/1 [==============================] - 0s 17ms/step
No faces found in 10.little-girl-with-messy-face-feria-de-mataderos-buenos-aires-argentina-g1k6g1.jpg, skipping to next image
1/1 [==============================] - 0s 17ms/step
No faces found in 10.peruvian-girl-face-pretty-young-teen-127625575.jpg, skipping to next image
2/2 [==============================] - 0s 3ms/step
No faces found in 10.salvador-mariona-el-salvadorjpg.jpg, skipping to next image
1/1 [==============================] - 0s 18ms/step
No faces found in 11.58bf31e1fa2607f8553f01fe843d8962.jpg, skipping to next image
1/1 [==============================] - 0s 17ms/step
No faces found in 11.7614186772_c2ae80b68b_b.jpg, skipping to next image
1/1 [==============================] - 0s 16ms/step
No faces found in 11.dfe77942683b6ae6bbc5a7db56269644.jpg, skipping to next image
1/1 [==============================] - 0s 17ms/step
No faces found in 12.bartomeu-melia.jpg, skipping to next image
1/1 [============================